## Correction of bollinger bands channel by RSX indicator

In [1]:
import pandas as pd
import numpy as np
from simple.chart import chartParallel, interactTable
from simple.funcs import vwap
from simple.jurik import JRSX
from simple.geneopt import GeneOpt
from multiprocessing import current_process
from simple.backtest import getLong, getShort, getProfit, npBacktestLimit

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)
len(T)

889360

In [3]:
# declare chart linestyles
line_styles = {
    'Tick': dict(color='gray', opacity=0.25),
    'Center': dict(color='blue', opacity=0.5),
    'qA': dict(color='red', opacity=0.5, dash='dot'),
    'qB': dict(color='green', opacity=0.5, dash='dot'),
    'cA': dict(color='red', opacity=0.4),
    'cB': dict(color='green', opacity=0.4),
    'RSX': dict(color='orange', row=2),
    
    'Profit': dict(color='gray', width=3, opacity=0.4, secondary_y=True, shape='hv', connectgaps=True),
    'Buy': dict(mode='markers', color='green', symbol='triangle-up', size=10, line=dict(color="darkgreen", width=1)),
    'Sell': dict(mode='markers', color='red', symbol='triangle-down', size=10, line=dict(color="darkred", width=1))
}

In [4]:
def model(Period: int = (1000, 50000), StdDev: float = (1, 4, 0.1), Threshold: int = (0, 20), Ratio: float = (0, 10, 0.1)):
    if Period == 0 or StdDev == 0: return {}
    Tick = T.Price
    Center = vwap(T, Period)
    std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + std * StdDev
    qB = Center - std * StdDev
    RSX = JRSX(Tick, Period) - 50
    ask_correction = Threshold - RSX
    bid_correction = -Threshold - RSX
    cA = qA + ask_correction * Ratio
    cB = qB + bid_correction * Ratio

    trades = npBacktestLimit(T, cA, cB)
    Buy, Sell = getLong(trades), getShort(trades)
    P = getProfit(trades)
    Profit = {'x': P.Index, 'y': P.Profit.cumsum()}

    return {
        'Profit': P.Profit.sum(),
        'Count': len(P),
        'AvgProfit': P.Profit.mean() if len(P) > 0 else 0,
        'Sharpe': P.Profit.sum() / P.Profit.std() if len(P) > 1 else 0
     } if current_process().daemon else locals()

In [5]:
# Genetic optimizer
G = GeneOpt(model)
G.maximize(population_size=300, generations=5)

  0%|          | 0/5 [00:00<?, ?it/s]

{'Period': 31078,
 'StdDev': 2.1244739954419254,
 'Threshold': 13,
 'Ratio': 3.497412414909004}

In [6]:
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Profit', ascending=False).set_index('Period')
X

,StdDev,Threshold,Ratio,Profit,Count,AvgProfit,Sharpe
Period,,,,,,,
31078,2.124474,13,3.497412,7380.908278,22.0,335.495831,15.880248
27881,2.296358,8,2.186412,7374.228434,23.0,320.618628,15.605568
31078,2.124474,8,5.720396,7207.946773,22.0,327.633944,15.309927
26846,2.388871,5,0.596195,7203.476395,23.0,313.194626,15.290129
33795,3.178509,8,2.228863,7118.257910,4.0,1779.564477,6.419886
...,...,...,...,...,...,...,...
0,3.232973,2,2.176262,NaN,NaN,NaN,NaN
0,2.302842,5,1.612309,NaN,NaN,NaN,NaN
31250,0.000000,19,2.356385,NaN,NaN,NaN,NaN


In [7]:
# Optimization results browser
interactTable(model, X, height=600, rows=2, **line_styles)

In [8]:
chartParallel(X)

FigureWidget({
    'data': [{'dimensions': [{'label': 'Period',
                              'range': [0, 49885],
                              'values': array([31078, 27881, 31078, ..., 31250, 33795, 31078])},
                             {'label': 'StdDev',
                              'range': [0.0, 3.991997430310988],
                              'values': array([2.124474  , 2.29635832, 2.124474  , ..., 0.        , 0.        ,
                                               0.        ])},
                             {'label': 'Threshold',
                              'range': [0, 19],
                              'values': array([13,  8,  8, ..., 19, 19, 10])},
                             {'label': 'Ratio',
                              'range': [0.0, 9.93873450042318],
                              'values': array([3.49741241, 2.18641194, 5.72039577, ..., 2.35638498, 0.        ,
                                               3.49741241])},
                             {'labe